<a href="https://colab.research.google.com/github/aofekiko/Cars196-E2E-CNN/blob/main/cars196.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project - Basics of deep learning
Hello dear students,<br> This is the template notebook. Please copy it into your drive.

---
<br>

### Name and ID:
Student 1:
<br>
Student 2:

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/20733/logos/header.png?t=2020-05-14-08-44-45">

# Utils

## Import Libraries

In [1]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import load_img
from keras.preprocessing import image
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1DS_5kLbzYYzMWtp1WP3x_hxtRBhroJID

## Dataset
Cars196 - 196 different classes of vehicles

### Download Data

In [2]:
%%capture
cars196 = tfds.load('Cars196', as_supervised=True, shuffle_files=True, with_info=True, data_dir=".")

Dictionary of the labels - maps between the label (int) number and the vehicle model (str)

In [3]:
label_dic = pd.read_csv('labels_dic.csv', header=None, dtype={0: str}).set_index(0).squeeze().to_dict()

### Function - Plot Single Example
This function receives an image and a label and it will display it on a plot

In [4]:
def plot_single_example(image, label, label_dic=label_dic):
  car_model_by_label = label_dic[str(label)]
  plt.title(f'Image Label: {car_model_by_label} ({label})')
  plt.imshow(image)


# Data Preprocessing

### Split Dataset
Train set contains 8041 examples<br>
Test set contains 8144 examples<br>
**It is allowed to change the ratio between the data sets.**


In [5]:
#cars_train=cars196['train']
#cars_test=cars196['test']

#[cars_train, cars_test], cars_info = tfds.load('Cars196',  split=["train", "test"], as_supervised=True, shuffle_files=True, with_info=True, data_dir=".")

[cars_train, cars_test] = tfds.load('Cars196',  split=["train[:90%]+test[:90%]", "train[90%:100%]+test[90%:100%]"], as_supervised=True, shuffle_files=True, data_dir=".")
print(len(cars_test))
print(len(cars_train))

1618
14567


Resize the data

In [6]:
height, width = 256, 256  # Previously: 256, 256 # Inputs divisable by 16 or 8 for better tensor core efficiency (https://www.tensorflow.org/guide/gpu_performance_analysis#1_enable_mixed_precision)

#def resize_and_rescale(image, label):
#  image = tf.cast(image, tf.float32)
#  image = tf.image.resize(image, [height, width])
#  image = (image / 255.0)
#  return image, label

def one_hot_encode(image, label):
  label = tf.one_hot(label, 196)
  return image, label


#size = (height, width)
#cars_train = cars_train.map(resize_and_rescale).map(one_hot_encode)
#cars_test = cars_test.map(resize_and_rescale).map(one_hot_encode)

# Parallel mapping to get rid of the input performance bottleneck
cars_train = cars_train.map(lambda x, y: (tf.image.resize_with_pad(x, height, width), y),num_parallel_calls=tf.data.AUTOTUNE)
cars_test = cars_test.map(lambda x, y: (tf.image.resize_with_pad(x, height, width), y),num_parallel_calls=tf.data.AUTOTUNE)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


### Example
Random example from the data set

In [7]:
#image, label = cars_train.as_numpy_iterator().next()
#plot_single_example(image, label)

#tfds.visualization.show_examples(cars_train, cars_info)

In [8]:
batch_size = 64 #64

#def augment_func(image, label):
#    image = tf.image.resize_with_crop_or_pad(image, height + 6, width + 6)
#    image = tf.image.random_crop(image, size=[height, width, 3])
#    image = tf.image.random_flip_left_right(image)
#    image = tf.image.random_hue(image, 0.2)
#    image = tf.image.random_contrast(image, 0.5, 2)
#    image = tf.image.random_saturation(image, 0, 2)
#    return image, label



#cars_train = cars_train.cache().map(augment_func).shuffle(100).batch(batch_size).prefetch(buffer_size=10)
cars_train = cars_train.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
cars_test = cars_test.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [9]:

# Mixed precision for better performance (https://www.tensorflow.org/guide/mixed_precision)
# With AMP: 
#   Test 1 - 295s
#   Test 2 - 388s
#   Test 3 - 330s
#   Avg - 337.6
# Without AMP = 
#   Test 1 - 288s
#   Test 2 - 500s
#   Test 2 - 494s
#   Avg - 427
tf.keras.mixed_precision.set_global_policy('mixed_float16')

model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1. / 255, input_shape=(height, width, 3)),
        tf.keras.layers.RandomFlip("horizontal"), #NEW
        tf.keras.layers.RandomRotation(0.2), #NEW
        tf.keras.layers.RandomContrast(0.2), #NEW
        #tf.keras.layers.RandomCrop(height, width, 3), #NEW Dataset does not include cropped or uncentered view of cars
        tf.keras.layers.RandomZoom(0, .2), #NEW
        tf.keras.layers.RandomBrightness(0.2), # NEW
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        #tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'), #NEW
        #tf.keras.layers.BatchNormalization(), #NEW
        #tf.keras.layers.MaxPooling2D(), #NEW
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dropout(0.5),
        #tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(196, activation='softmax', dtype='float32') # Force datatype to be 32 due to enabling mixed precision (Nvidia AXP)
    ])

##https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide/notebook

#model=tf.keras.models.Sequential([
#    tf.keras.layers.Rescaling(1. / 255, input_shape=(height, width, 3)),
#    tf.keras.layers.RandomFlip("horizontal"),
#    tf.keras.layers.RandomRotation(0.2),
#    tf.keras.layers.RandomContrast(0.5, 2),
#    #tf.keras.layers.RandomCrop(height, width, 3),
#    tf.keras.layers.RandomZoom(.5, .2),
#    tf.keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu'),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
#    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.MaxPool2D(pool_size=(3,3)),
#    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(256,activation='relu'),
#    #tf.keras.layers.Dropout(0.5),
#    tf.keras.layers.Dense(256,activation='relu'),
#    #tf.keras.layers.Dropout(0.5),
#    tf.keras.layers.Dense(196,activation='softmax')  
#])

#
#model = tf.keras.models.Sequential([
#tf.keras.layers.Rescaling(1. / 255, input_shape=(height, width, 3)),
##tf.keras.layers.Input(shape=(height,width,3)),
##tf.keras.layers.RandomFlip("horizontal")
##tf.keras.layers.RandomRotation(0.2)
##tf.keras.layers.RandomContrast(0.5, 2)
##tf.keras.layers.RandomCrop(height, width, 3)
##tf.keras.layers.RandomZoom(.5, .2)
#tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, padding='valid', activation='relu'),
#tf.keras.layers.MaxPool2D(pool_size=3, strides = 2, padding='valid'),
#tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding="same", activation='relu'),
#tf.keras.layers.MaxPool2D(pool_size=3, strides = 2, padding='valid'),
#tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding="same", activation='relu'),
#tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding="same", activation='relu'),
#tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation='relu'),
#tf.keras.layers.MaxPool2D(pool_size=3, strides = 2, padding='valid'),
#tf.keras.layers.Flatten(),
##tf.keras.layers.Dense(4096,activation="relu"),
##tf.keras.layers.Dense(4096,activation="relu"),
#tf.keras.layers.Dense(1000,activation="relu"),
#tf.keras.layers.Dense(196,activation="softmax"),
#])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 random_flip (RandomFlip)    (None, 256, 256, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 256, 256, 3)      0         
 tion)                                                           
                                                                 
 random_contrast (RandomCont  (None, 256, 256, 3)      0         
 rast)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 256, 256, 3)       0         
                                                                 
 random_brightness (RandomBr  (None, 256, 256, 3)      0

In [10]:


from datetime import datetime
from packaging import version

import os

run_name="BetterAugmentation"

logs = "logs/" + run_name + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs, profile_batch=(64))

#tf.config.optimizer.set_jit(True) enables auto-clustering XLA https://www.tensorflow.org/xla#auto-clustering
#print(tf.config.optimizer.get_jit())

#Gets an error: 
#CUDNN_STATUS_INTERNAL_ERROR
#in tensorflow/stream_executor/cuda/cuda_dnn.cc(4369): 'status'
#	 [[{{node cluster_3_1/xla_run}}]] [Op:__inference_train_function_6519]

In [ ]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=40) # start_from_epoch=50 Missing feature in ts2.10, available in ts2.11+

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=10000,
        decay_rate=0.9)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
       filepath='./weights/cp-{epoch:04d}.ckpt',
        save_weights_only=True,
        save_freq='epoch')

#Sparese is better than normal cross entropy in our case https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01,momentum=0.9),loss='sparse_categorical_crossentropy' ,metrics=['accuracy']) 

history = model.fit(x = cars_train, validation_data = cars_test, epochs = 500, callbacks=[earlystop_callback, tboard_callback, cp_callback], batch_size=batch_size, verbose=1)

from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Epoch 1/500


227/227 [==============================] - 310s 1s/step - loss: 7.6496 - accuracy: 0.0044 - val_loss: 5.9998 - val_accuracy: 0.0088
Epoch 2/500
227/227 [==============================] - 268s 1s/step - loss: 5.9259 - accuracy: 0.0062 - val_loss: 5.3948 - val_accuracy: 0.0075
Epoch 3/500
227/227 [==============================] - 272s 1s/step - loss: 5.4675 - accuracy: 0.0057 - val_loss: 5.3077 - val_accuracy: 0.0069
Epoch 4/500
227/227 [==============================] - 275s 1s/step - loss: 5.3438 - accuracy: 0.0054 - val_loss: 5.2909 - val_accuracy: 0.0044
Epoch 5/500
227/227 [==============================] - 268s 1s/step - loss: 5.3001 - accuracy: 0.0066 - val_loss: 5.2843 - val_accuracy: 0.0088
Epoch 6/500
227/227 [==============================] - 267s 1s/step - loss: 5.2868 - accuracy: 0.0065 - val_loss: 5.2790 - val_accuracy: 0.0088
Epoch 7/500
115/227 [==============>...............] - ETA: 2:08 - loss: 5.2791 - accuracy: 0.0068